# Overview

Once the necessary libraries are imported, the following actions are performed:

- Load Labelled Data:<br>
Here we have the PDF names (or Data IDs) and we also have manually marked which pages on these PDFs are maps. First, we create the dataframe of the features by extracting the features of each page on these PDFs using the function "extract_features". Then by using the marked values of each page being map or not we create the dataframe for the dependent variable. The variable "dataID_pageNo" is onnly used to identify a certain page. 
- Train Test Split: <br>

In [1]:
import pandas as pd

import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
from feature_extraction import extract_features

# Load Labeled Data 

In [3]:
path_pdf = "C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\RF Training Set\\"

DataIDHand = [268712,  486221, 500633, 
               555093, 684494, 
              895015, 2392922, 2445549, 2758927,
              2813701,  
              2967854, 2968069,  
              3891802,
              4036098]
Pages = [[3,4,5,8,9,10,14,15,24,25,26], range(1,5),  [5,9], 
          [6,9,33,34], [12,13,14],
         range(1,11), [1], range(1,4),  [9],
         [40, 92, 95, 143, 170, 180, 216, 217, 218, 219],
         [3,4],range(1,13),  
         [33, 34, 35, 89, 90, 91, 92, 93, 100, 146, 147, 148, 149, 153, 154, 159, 160, 161, 162, 165, 166, 169, 170, 173, 174, 177, 178, 181, 182, 184, 185, 188, 189], 
          []]

print(len(DataIDHand))
print(len(Pages))

14
14


In [4]:
# #fetching featuresfor the pages of the PDF Files
# X_df, dataIDs, error_files = extract_features(DataIDHand, path_pdf) 
# #Features
# #dataIDs
# #error_files

In [ ]:
# X_df.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_test_train.csv")
# dataIDs.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs.csv")
# print(len(error_files))

In [ ]:
X_df = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_test_train.csv", index_col = 0)
dataIDs = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs.csv", index_col = 0)
X_df.head()

In [ ]:
X_df_features = X_df.copy()
X_df_features.drop(columns=['dataID_pageNo'], inplace=True)
X_df_features.head()

In [ ]:
def get_Y_values(dataIDs, Pages):
    Y_class = []
    dataID_pageNo = []
    j = 0
    for index, row in dataIDs.iterrows():
        #print(row['DataIDs'])
        #print(row['Page_no'])
        for i in range(1,row['Page_no']+1):
            if i in Pages[j]:
                Y_class.append(1)
            else:
                Y_class.append(0)
            dataID_pageNo.append(str(row['DataIDs']) + "_" +str(i))
        j = j+1
    
    Y_df = pd.DataFrame({'dataID_pageNo' : dataID_pageNo, 
                         'Y_class' : Y_class})
    Y_dfclass = pd.DataFrame({'Y_class' : Y_class})
    
    return Y_df, Y_dfclass
    
                
Y_df, Y_dfclass = get_Y_values(dataIDs, Pages)

In [ ]:
print(len(Y_df))
print(len(X_df))
print(len(Y_dfclass))

# Preparing Data for Training 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df_features,
                                                    Y_dfclass,
                                                    test_size = 0.25,
                                                    random_state = 8)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
print("Training Set: ", len(y_train))
print("Alignment Sheets in Training Set: ", len(y_train[y_train.Y_class > 0]))
print()
print("Test Set: ", len(y_test))
print("Alignment Sheets in Training Set: ", len(y_test[y_test.Y_class > 0]))

# Implement Classification Models

In [ ]:
import random

random.seed(19)

classifiers = []
name = []
# we will create an array of Classifiers and append different classification models to our array.
model1 = xgboost.XGBClassifier()
classifiers.append(model1)
name.append("xgboost")

model2 = svm.SVC()
classifiers.append(model2)
name.append("svc")

model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)
name.append("decisiontree")

model4 = RandomForestClassifier()
classifiers.append(model4)
name.append("rfc")


model5 = RandomForestRegressor(n_estimators=5)
classifiers.append(model5)
name.append("rfr5")

model6 = RandomForestRegressor(n_estimators=25)
classifiers.append(model6)
name.append("rfr25")

model7 = RandomForestRegressor(n_estimators=50)
classifiers.append(model7)
name.append("rfr50")

model8 = RandomForestRegressor(n_estimators=75)
classifiers.append(model8)
name.append("rfr75")

model9 = RandomForestRegressor(n_estimators=100)
classifiers.append(model9)
name.append("rfr100")


model10 = XGBRegressor(n_estimators=5)
classifiers.append(model10)
name.append("xgbr5")

model11 = XGBRegressor(n_estimators=25)
classifiers.append(model11)
name.append("xgbr25")

model12 = XGBRegressor(n_estimators=50)
classifiers.append(model12)
name.append("xgbr50")

model13 = XGBRegressor(n_estimators=75)
classifiers.append(model13)
name.append("xgbr75")

model14 = XGBRegressor(n_estimators=100)
classifiers.append(model14)
name.append("xgbr100")

In [ ]:
i = 0
random.seed(10)
test_accuracy = []
for clf in classifiers:
    print("________________________________________________________")
    print("________________________________________________________")
    #fit our algorithms in our Train dataset 
    clf.fit(X_train, y_train)
    
    #get test dataset prediction
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_test)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_test)
        
    print(name[i])
    acc = accuracy_score(y_test, y_pred)
    test_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    i = i +1

# Testing with Test Dataset

In [ ]:
DataIDHand = [3410189, 3970828, 2968356]
Pages = [[], 
         [29, 35, 51, 59, 100, 101, 108, 109, 165, 179, 225, 231, 293, 294], 
         [9,18, 26]]

print(len(DataIDHand))
print(len(Pages))

In [ ]:
# path_pdf = "C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\RF Validation Set\\"

# # #fetching featuresfor the pages of the PDF Files
# X_df_valid, dataIDs_valid, error_files = extract_features(DataIDHand, path_pdf) 
# # #Features
# # #dataIDs
# # #error_files

In [ ]:
# X_df_valid.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_valid.csv")
# dataIDs_valid.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs_valid.csv")
# print(len(error_files))

In [ ]:
X_df_valid = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_valid.csv", index_col = 0)
dataIDs_valid = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs_valid.csv", index_col = 0)
X_df_valid.head()

In [ ]:
X_df_features_valid = X_df_valid.copy()
X_df_features_valid.drop(columns=['dataID_pageNo'], inplace=True)
X_df_features_valid.head()

In [ ]:
Y_df_valid, Y_dfclass_valid = get_Y_values(dataIDs_valid, Pages)

print(len(Y_df_valid))
print(len(X_df_features_valid))
print(len(X_df_valid))
print(len(Y_dfclass_valid))

In [ ]:
i = 0
random.seed(10)
test_accuracy = []
valid_accuracy = []
cm_test = []
cm_valid = []
for clf in classifiers:
    print("________________________________________________________")
    print("________________________________________________________")
    #fit our algorithms in our Train dataset 
    clf.fit(X_train, y_train)
    
    #get test dataset prediction
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_test)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_test)
        
    print(name[i])
    acc = accuracy_score(y_test, y_pred)
    test_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    cm_test.append(cm)
    
    
    print("________________Validation Set ___________________________")
    #get validation accuracy
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_df_features_valid)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_df_features_valid)
        
    print(name[i])
    acc = accuracy_score(Y_dfclass_valid["Y_class"], y_pred)
    valid_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(Y_dfclass_valid["Y_class"], y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    cm_valid.append(cm)
    i = i +1

In [ ]:
classification_models = pd.DataFrame({'name': name, 
                                     'test_accuracy': test_accuracy,
                                     'test_cm': cm_test, 
                                     'valid_accuracy':valid_accuracy,
                                     'valid_cm': cm_valid})
classification_models["product"] = classification_models["test_accuracy"]*classification_models["valid_accuracy"]

classification_models = classification_models.sort_values(by=['product'])
classification_models.head(15)

In [ ]:
i = 0
random.seed(10)

for clf in classifiers:
    
    if name[i] != "rfc":
        i = i +1
        continue
    print(name[i])
    clf.fit(X_train, y_train)
    filename = "G:\\ESA_downloads\\Demo_Alignment_Sheets\\alignment_sheet_classifier_rfc.sav"
    pickle.dump(clf, open(filename, 'wb'))
    
    filename = "alignment_sheet_classifier_rfr50.sav"
    pickle.dump(clf, open(filename, 'wb'))
    i = i +1

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=50, random_state=0)
#n_estimators - Number of trees in the forest 
#random_state - Controls both the randomness of the bootstrapping of the samples used when building 
              # trees (if bootstrap=True) and the sampling of the features to consider when looking 
              # for the best split at each node (if max_features < n_features)
# Ref -> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


regressor.fit(X_train, y_train)

file_name =  "alignment_sheet_classifier_rfr.sav"
pickle.dump(regressor, open(file_name, 'wb'))

y_pred = regressor.predict(X_test)
#y_pred.shape
#y_pred
y_predict = []
for y in y_pred:
    if y > 0.40:
        y_predict.append(1)
    else:
        y_predict.append(0)
#y_predict


In [ ]:
f_importance = regressor.feature_importances_
feature = []

for col in X_df_features:
    feature.append(col)
    
df_f_importance = pd.DataFrame({'Feature_Name' :  feature, 
                                'Importance':  f_importance})
df_f_importance

In [ ]:
# https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))
print(accuracy_score(y_test, y_predict))

In [ ]:
XYRdf_dataset = X_test
XYRdf_dataset['Y_test'] = y_test
XYRdf_dataset['y_score'] = y_pred
XYRdf_dataset['y_predict'] = y_predict
#XYRdf_dataset['dataID_pageNo' : dataID_pageNo]
len(XYRdf_dataset)
XYRdf_dataset.head()


XYRdf_dataset.to_csv(path_pdf + "Results_Training_Set.csv")